In [1]:
import multiprocessing
import threading
import tensorflow as tf
from ACNet import Access
from Framework import Framework

In [2]:
NUMS_CPU = multiprocessing.cpu_count()
state_size = 58
batch_size = 50
action_size = 3
max_episodes = 10
GD = {}

In [3]:
class Worker(Framework):

    def __init__(self, name, access, state_size, batch_size, action_size):
        super().__init__(name, access, state_size, batch_size, action_size)

    def run(self, sess, max_episodes, t_max=8):
        episode_score_list = []
        episode = 0
        while episode < max_episodes:
            episode += 1
            episode_socre, _ = self.run_episode(sess, t_max)
            episode_score_list.append(episode_socre)
            GD[str(self.name)] = episode_score_list
            if self.name == 'W0':
                print('Episode: %f, score: %f' % (episode, episode_socre))
                print('\n')

In [4]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# with tf.Session(config=config) as sess:
with tf.Session() as sess:
    with tf.device("/cpu:0"):
        A = Access(state_size, batch_size, action_size)
        F_list = []
        for i in range(NUMS_CPU):
            F_list.append(Worker('W%i' % i, A, state_size, batch_size, action_size))
        COORD = tf.train.Coordinator()
        sess.run(tf.global_variables_initializer())
        sess.graph.finalize()

        threads_list = []
        for ac in F_list:
            job = lambda: ac.run(sess, max_episodes)
            t = threading.Thread(target=job)
            t.start()
            threads_list.append(t)
        COORD.join(threads_list)
        A.save(sess, 'model/saver_1.ckpt')

E:\software\Anaconda363\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


graph W0
graph W1
graph W2
graph W3
graph W4
graph W5
graph W6
graph W7
actor: -4.351710, actor_grad: 4.918072, policy mean: 1.109558, policy: -4.340945, entropy: -0.010765, critic: 0.046404, critic_grad: 0.432405, value: 0.046404, value_mean: -0.316479, advantage: -3.578690
Episode: 1.000000, score: 0.788775


actor: -3.779090, actor_grad: 1.477806, policy mean: 0.953492, policy: -3.769613, entropy: -0.009478, critic: 0.003799, critic_grad: 0.211271, value: 0.003799, value_mean: -0.083649, advantage: -4.002062
Episode: 2.000000, score: 0.795353


actor: 10.473018, actor_grad: 6.649261, policy mean: 1.003981, policy: 10.482761, entropy: -0.009743, critic: 0.107355, critic_grad: 22.453459, value: 0.107355, value_mean: -0.080331, advantage: 10.447111
Episode: 3.000000, score: 0.845088


actor: -0.638486, actor_grad: 19.069897, policy mean: 0.790759, policy: -0.630594, entropy: -0.007892, critic: 1.570231, critic_grad: 1078.013062, value: 1.570231, value_mean: 0.221685, advantage: -2.3715